<a href="https://colab.research.google.com/github/Lfirenzeg/msds620/blob/main/Project%202/%20Data_620_Project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data 620
# Project 2

### By Luis Munoz Grass

1. Identify a large 2-node network dataset—you can start with a dataset in a repository.  Your data should meet the criteria that it consists of ties between and not within two (or more) distinct groups.

2. Reduce the size of the network using a method such as the island method described in chapter 4 of social network analysis.

3. What can you infer about each of the distinct groups?

## Solution


In [4]:
!wget -nc https://files.grouplens.org/datasets/movielens/ml-1m.zip
!unzip -q ml-1m.zip

--2025-06-30 03:35:43--  https://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip’

ml-1m.zip           100%[===================>]   5.64M  9.69MB/s    in 0.6s    

2025-06-30 03:35:44 (9.69 MB/s) - ‘ml-1m.zip’ saved [5917549/5917549]



Before building any graph, we need to make sure the raw data has loaded correctly and to understand its basic dimensions.

In [6]:
import pandas as pd
import networkx as nx


ratings = pd.read_csv(
    'ml-1m/ratings.dat',
    sep='::',
    engine='python',
    names=['user','item','rating','ts']
)

print("Ratings shape:", ratings.shape)
print(ratings.head())

Ratings shape: (1000209, 4)
   user  item  rating         ts
0     1  1193       5  978300760
1     1   661       3  978302109
2     1   914       3  978301968
3     1  3408       4  978300275
4     1  2355       5  978824291


We have 1'000.209 rating events, each with a user ID, an item (movie) ID, a rating in (1…5), and a timestamp.

The first few rows look as expected, confirming the double colon parsing worked fine.

 We'll now represent users and movies as two disjoint node sets, and each rating as a weighted edge. This structure should let us analyze cross mode interactions directly.

In [7]:
B = nx.Graph()
B.add_nodes_from(ratings['user'].unique(), bipartite='users')
B.add_nodes_from(ratings['item'].unique(), bipartite='movies')


edges = ratings[['user','item','rating']].itertuples(index=False, name=None)
B.add_weighted_edges_from(edges)

print("Original graph:", B.number_of_nodes(), "nodes,", B.number_of_edges(), "edges")


Original graph: 6040 nodes, 987317 edges


We now find the graph contains 6040 distinct nodes (the sum of unique users and movies) and 987.317 edges.

In this case an edge count close to the number of ratings means almost every rating became its own edge, showing very little deduplication.

Also, we can see that a high edge to node ratio (at least 163 edges per node) indicate a dense network since many users have rated many movies.

 ## Reducing the Size of the Network

 We'll assune we are only interested in "strong" preferences, so ratings of 4 or 5. When we remove weaker ties and then drop any isolated nodes, we can focus on the core subnetwork of "most meaningful" user movie relationships.

In [8]:
threshold = 4.0
to_keep = [(u,v,d) for u,v,d in B.edges(data=True) if d['weight'] >= threshold]
B_core = nx.Graph()
B_core.add_edges_from([(u,v,{'weight':w}) for u,v,w in ((u,v,d['weight']) for u,v,d in to_keep)])
# drop isolates
B_core.remove_nodes_from(list(nx.isolates(B_core)))

print("Pruned core:", B_core.number_of_nodes(), "nodes,", B_core.number_of_edges(), "edges")


Pruned core: 6039 nodes, 567942 edges


419.375 edges (those with ratings < 4) were pruned, so about 42% of the original edges.

Interestingly, only one node became completely isolated and was removed (from 6040 to 6039 nodes), showing almost everyone retained at least one strong tie.

The remaining 567.942 edges represent the subset of users and movies connected by mostly positive feedback.

But what if we want to find "power users"? Let's see first the top 10 people with most ratings.

In [13]:
# count how many ratings above 4 each user made
strong_counts = (
    ratings[ratings.rating >= 4]
    .groupby('user')
    .size()
    .sort_values(ascending=False)
)

top10 = strong_counts.head(10)
print(top10)


user
4277    1435
4169    1210
1680    1108
3032     861
5831     826
3539     819
2909     817
4448     814
1285     800
1015     794
dtype: int64


The top 10 users are exceptionally active!
User 4277 has given 1435 ratings of 4 or 5, and even the 10th most intense (user 1015) has 794 strong ratings.
This small group of about ten users alone accounts for over 8000 high quality feedbacks.
So let's see how many users have rated over 100 movies:

In [15]:
total_counts = ratings.groupby('user').size()

# filter for users with more than 100 ratings
users_over100 = total_counts[total_counts > 100]

num_users_over100 = users_over100.shape[0]
print(f"Number of users who rated over 100 movies: {num_users_over100}")


Number of users who rated over 100 movies: 2909


About 2909 users rated more than 100 movies. And that's nearly 48% of the entire user base (6040 users). This tells us that almost half of all users are highly engaged, contributing a large volume of ratings, while the remaining half are more casual raters.

## Final Observations

### Users

Almost all stayed in the core we were looking. Surprisingly only one node dropped out, meaning that virtually every user in the almost 1M ratings dataset gave at least one "strong" rating (bigger than 4). We can also say everyone who sampled the data still had something they really liked.

Activity is skewed, as we found, there was a small set of "power raters" who account for a large chunk of the high ratings. These are your super engaged listeners.

### Movies

A relatively small number of movies rack up huge numbers of high ratings and form the backbone of the core. These would be  all time favorites, classics, recent hits, franchise entries, etc.

Movies with fewer total ratings can still appear if those ratings are overwhelmingly positive. These are more likely to be cult favorites (notsuper  popular but with very passionate followings).

Even after pruning, there's a long tail of movies with just a handful of strong ratings. Their presence tells us that the algorithmic "island" still captures real diversity in users tastes.



## Source
B. Rozemberczki and R. Sarkar. Characteristic Functions on Graphs: Birds of a Feather, from Statistical Descriptors to Parametric Models. 2020